In [1]:
import numpy as np
import networkx as nx

from example_stats import d_connected_triplet_motif_density

In [22]:
G = nx.DiGraph(nx.scale_free_graph(100))
adj = nx.convert_matrix.to_scipy_sparse_matrix(A)
print(adj.toarray())

[[1 9 1 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 [2 3 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]


In [2]:
# Utility functions for manipulating motif counts
def directed_triplet_motif_index(G):
    """Return the directed motif index of three-node graph G (G is a nx graph type)
    
    The motif index is then computed as follows:
    Each possible (undirected) edge on the nodes of G is sorted in lexicographic order.
    For each pair of vertices, two bits encode, in order, the presence of the edge from
    lower index to higher index and the edge from higher index to lower index. These bits
    are reversed and concatenated to form a single integer
    
    Example: G has three nodes, labeled i,j,k in sorted order. It has edge ij, ik, ki, and kj.
    The lex order for the pairs is ij, ik, jk. Pair ij has edge ij (low-high) but not ji (high-low),
    so the least significant bit is 1 and the second-least significant bit is 0. For pair ik, we have both
    directed edges so those bits are 11. Lastly, pair jk has only the high-low edge, so the higher-order
    bit is 1 while the lower bit is 0. Putting these bits together from right to left we get 101101,
    which is 45 in decimal.
    
    !!! Note that the order of vertice s in G has nothing to do with numerical order!
        See networkx documentation about classes OrderedGraph and OrderedDiGraph.
    
    Returns an integer between 0 and 63 (inclusive)
    """
    bit_selector = np.array([[0,1,4], [2,0,16], [8, 32, 0]])
    return np.sum(np.multiply(bit_selector, nx.to_numpy_matrix(G).astype(int)))

def directed_triplet_motif_index_from_matrix(M):
    """Same as directed_triplet_motif_index but accepts a numpy matrix as its argument"""
    bit_selector = np.array([[0,1,4], [2,0,16], [8, 32, 0]])
    return np.sum(np.multiply(bit_selector, M))


def binary_digits(n, d):  # numpy-optimized
    """Returns an n x d array of the binary digits of each entry of array n
    Parameters:
        n : array_like
            Integer values to be represented as binary digits
        d : the number of digits; zero padding and/or truncation if necessary
    Returns:
        digits : an n x d binary array; each row is the digits of the corresponding entry of n. Least significant bit has index 0.
    """
    return ((n[:, None] & (1 << np.arange(d))) > 0).astype(int)

def index_to_directed_triplet_motif_matrix(n):
    """Return the adjacency matrix corresponding to motif with index n, as defined by the function
    directed_triplet_motif_index"""
    digs = binary_digits(np.array([n]),6)
    A = np.zeros((3,3), dtype=int)
    A[tuple([[0,1,0,2,1,2],[1,0,2,0,2,1]])] = digs
    return A

In [3]:
for k in range(64):
    print(k, index_to_directed_triplet_motif_matrix(k))

0 [[0 0 0]
 [0 0 0]
 [0 0 0]]
1 [[0 1 0]
 [0 0 0]
 [0 0 0]]
2 [[0 0 0]
 [1 0 0]
 [0 0 0]]
3 [[0 1 0]
 [1 0 0]
 [0 0 0]]
4 [[0 0 1]
 [0 0 0]
 [0 0 0]]
5 [[0 1 1]
 [0 0 0]
 [0 0 0]]
6 [[0 0 1]
 [1 0 0]
 [0 0 0]]
7 [[0 1 1]
 [1 0 0]
 [0 0 0]]
8 [[0 0 0]
 [0 0 0]
 [1 0 0]]
9 [[0 1 0]
 [0 0 0]
 [1 0 0]]
10 [[0 0 0]
 [1 0 0]
 [1 0 0]]
11 [[0 1 0]
 [1 0 0]
 [1 0 0]]
12 [[0 0 1]
 [0 0 0]
 [1 0 0]]
13 [[0 1 1]
 [0 0 0]
 [1 0 0]]
14 [[0 0 1]
 [1 0 0]
 [1 0 0]]
15 [[0 1 1]
 [1 0 0]
 [1 0 0]]
16 [[0 0 0]
 [0 0 1]
 [0 0 0]]
17 [[0 1 0]
 [0 0 1]
 [0 0 0]]
18 [[0 0 0]
 [1 0 1]
 [0 0 0]]
19 [[0 1 0]
 [1 0 1]
 [0 0 0]]
20 [[0 0 1]
 [0 0 1]
 [0 0 0]]
21 [[0 1 1]
 [0 0 1]
 [0 0 0]]
22 [[0 0 1]
 [1 0 1]
 [0 0 0]]
23 [[0 1 1]
 [1 0 1]
 [0 0 0]]
24 [[0 0 0]
 [0 0 1]
 [1 0 0]]
25 [[0 1 0]
 [0 0 1]
 [1 0 0]]
26 [[0 0 0]
 [1 0 1]
 [1 0 0]]
27 [[0 1 0]
 [1 0 1]
 [1 0 0]]
28 [[0 0 1]
 [0 0 1]
 [1 0 0]]
29 [[0 1 1]
 [0 0 1]
 [1 0 0]]
30 [[0 0 1]
 [1 0 1]
 [1 0 0]]
31 [[0 1 1]
 [1 0 1]
 [1 0 0]]
32 [[0 0 0]
 [0 0 

In [6]:
b = np.arange(400).reshape((20,20))
partition = {"A": (0,4), "B": (4,10), "C": (10,17), "D": (17,20)}

In [9]:
for c, r in partition.items():
    print(c, list(range(*r)))

A [0, 1, 2, 3]
B [4, 5, 6, 7, 8, 9]
C [10, 11, 12, 13, 14, 15, 16]
D [17, 18, 19]


In [10]:
b[partition["A"][0]:partition["A"][1], partition["B"][0]:partition["B"][1]]

array([[ 4,  5,  6,  7,  8,  9],
       [24, 25, 26, 27, 28, 29],
       [44, 45, 46, 47, 48, 49],
       [64, 65, 66, 67, 68, 69]])

In [46]:
b[np.ix_(partition["A"], partition["B"])]

array([[ 4,  5,  6,  7,  8,  9],
       [24, 25, 26, 27, 28, 29],
       [44, 45, 46, 47, 48, 49],
       [64, 65, 66, 67, 68, 69]])

In [51]:
%timeit sub_matrix = b[partition["A"],:][:,partition["B"]]

20.9 µs ± 1.66 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [49]:
%timeit sub_matrix = b[np.ix_(partition["A"], partition["B"])]

34.5 µs ± 3.35 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [11]:
%timeit sub_matrix = b[partition["A"][0]:partition["A"][1], partition["B"][0]:partition["B"][1]]

646 ns ± 14.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [63]:
b[[1:4, 5:7], [1:4, 5:7]]

SyntaxError: invalid syntax (<ipython-input-63-c9b768c89bb6>, line 1)

In [71]:
np.vstack([np.hstack([b[0:4, 0:4], b[0:4, 10:17]]), np.hstack([b[10:17, 0:4], b[10:17, 10:17]])])

array([[  0,   1,   2,   3,  10,  11,  12,  13,  14,  15,  16],
       [ 20,  21,  22,  23,  30,  31,  32,  33,  34,  35,  36],
       [ 40,  41,  42,  43,  50,  51,  52,  53,  54,  55,  56],
       [ 60,  61,  62,  63,  70,  71,  72,  73,  74,  75,  76],
       [200, 201, 202, 203, 210, 211, 212, 213, 214, 215, 216],
       [220, 221, 222, 223, 230, 231, 232, 233, 234, 235, 236],
       [240, 241, 242, 243, 250, 251, 252, 253, 254, 255, 256],
       [260, 261, 262, 263, 270, 271, 272, 273, 274, 275, 276],
       [280, 281, 282, 283, 290, 291, 292, 293, 294, 295, 296],
       [300, 301, 302, 303, 310, 311, 312, 313, 314, 315, 316],
       [320, 321, 322, 323, 330, 331, 332, 333, 334, 335, 336]])

In [4]:
m = index_to_directed_triplet_motif_matrix(7)
m

array([[0, 1, 1],
       [1, 0, 0],
       [0, 0, 0]])

In [5]:
if m[1,0] | m[0,1]:
    print("yep")
else:
    print("nop")

yep


In [6]:
m[0,1]

1

In [7]:
sum(m[i,j] | m[j,i] for i in range(3) for j in range(i,3))

2

In [34]:
c12 = m[0,1] | m[1,0]
c13 = m[0,2] | m[2,0]
c23 = m[1,2] | m[2,1]
print(c12, c13, c23)

1 1 0


In [39]:
for k in range(64):
    m = index_to_directed_triplet_motif_matrix(k)
    print(k, m)
    c12 = m[0,1] + m[1,0]
    c13 = m[0,2] + m[2,0]
    c23 = m[1,2] + m[2,1]
    if not c12 and not c13 and not c23:
        print("none of them!")
    elif c12 and not c13 and not c23:
        print("Only c12")
    elif not c12 and c13 and not c23:
        print("only c13")
    elif not c12 and not c13 and c23:
        print("only c23")
    elif c12 and c13 and not c23:
        print("c12 and c13 but not c23")
    elif c12 and not c13 and c23:
        print("c12 and c23, but not c13")
    elif not c12 and c13 and c23:
        print("missing c12")
    else:
        print("got 'em all")

0 [[0 0 0]
 [0 0 0]
 [0 0 0]]
none of them!
1 [[0 1 0]
 [0 0 0]
 [0 0 0]]
Only c12
2 [[0 0 0]
 [1 0 0]
 [0 0 0]]
Only c12
3 [[0 1 0]
 [1 0 0]
 [0 0 0]]
Only c12
4 [[0 0 1]
 [0 0 0]
 [0 0 0]]
only c13
5 [[0 1 1]
 [0 0 0]
 [0 0 0]]
c12 and c13 but not c23
6 [[0 0 1]
 [1 0 0]
 [0 0 0]]
c12 and c13 but not c23
7 [[0 1 1]
 [1 0 0]
 [0 0 0]]
c12 and c13 but not c23
8 [[0 0 0]
 [0 0 0]
 [1 0 0]]
only c13
9 [[0 1 0]
 [0 0 0]
 [1 0 0]]
c12 and c13 but not c23
10 [[0 0 0]
 [1 0 0]
 [1 0 0]]
c12 and c13 but not c23
11 [[0 1 0]
 [1 0 0]
 [1 0 0]]
c12 and c13 but not c23
12 [[0 0 1]
 [0 0 0]
 [1 0 0]]
only c13
13 [[0 1 1]
 [0 0 0]
 [1 0 0]]
c12 and c13 but not c23
14 [[0 0 1]
 [1 0 0]
 [1 0 0]]
c12 and c13 but not c23
15 [[0 1 1]
 [1 0 0]
 [1 0 0]]
c12 and c13 but not c23
16 [[0 0 0]
 [0 0 1]
 [0 0 0]]
only c23
17 [[0 1 0]
 [0 0 1]
 [0 0 0]]
c12 and c23, but not c13
18 [[0 0 0]
 [1 0 1]
 [0 0 0]]
c12 and c23, but not c13
19 [[0 1 0]
 [1 0 1]
 [0 0 0]]
c12 and c23, but not c13
20 [[0 0 1]
 [0 0 1]
 

In [37]:
?range

Init signature: range(self, /, *args, **kwargs)
Docstring:     
range(stop) -> range object
range(start, stop[, step]) -> range object

Return an object that produces a sequence of integers from start (inclusive)
to stop (exclusive) by step.  range(i, j) produces i, i+1, i+2, ..., j-1.
start defaults to 0, and stop is omitted!  range(4) produces 0, 1, 2, 3.
These are exactly the valid indices for a list of 4 elements.
When step is given, it specifies the increment (or decrement).
Type:           type
Subclasses:     


In [38]:
list(reversed(range(10)))

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [46]:
%timeit m[0,1] & m[1,0]

577 ns ± 7.81 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [47]:
%timeit m[0,1] * m[1,0]

593 ns ± 10.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [13]:
c1 = "A"
c2 = "B"
%timeit max(1, (c1 == c2) + m[0,1] + m[1,0] - 1)

4.5 µs ± 133 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [15]:
%timeit 1 + ((c1 == c2) & m[0,1] & m[1,0])

4.1 µs ± 132 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [14]:
%timeit ((c1 == c2) & m[0,1] & m[1,0]) + 1

4.11 µs ± 114 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
